In [1]:
import numpy as np
import pandas as pd

df_regular = pd.read_csv('DataFiles/RegularSeasonCompactResults.csv')
df_regular.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
0,1985,20,1228,81,1328,64,N,0
1,1985,25,1106,77,1354,70,H,0
2,1985,25,1112,63,1223,56,H,0
3,1985,25,1165,70,1432,54,H,0
4,1985,25,1192,86,1447,74,H,0


In [2]:
df_team_conference = pd.read_csv('DataFiles/TeamConferences.csv')
df_team_conference.head()

,Season,TeamID,ConfAbbrev
0,1985,1114,a_sun
1,1985,1147,a_sun
2,1985,1204,a_sun
3,1985,1209,a_sun
4,1985,1215,a_sun


We want to compute these features from regular season. Regular season includes conference tourney. We may want to compute the same features from conference tourney seperately as they'are good proxy of NCAA tourney.
1. Each team's winning percentage
2. Each team's average winning margin
3. Each conference's winning percentage
4. Each conference's average winning margin

In [3]:
df_win_team_conference = df_team_conference.rename(columns={'TeamID':'WTeamID', 'ConfAbbrev':'WTeamConfAbbrev'})
df_win_team_conference.head()

,Season,WTeamID,WTeamConfAbbrev
0,1985,1114,a_sun
1,1985,1147,a_sun
2,1985,1204,a_sun
3,1985,1209,a_sun
4,1985,1215,a_sun


In [4]:
df_loss_team_conference = df_team_conference.rename(columns={'TeamID':'LTeamID', 'ConfAbbrev':'LTeamConfAbbrev'})
df_loss_team_conference.head()

,Season,LTeamID,LTeamConfAbbrev
0,1985,1114,a_sun
1,1985,1147,a_sun
2,1985,1204,a_sun
3,1985,1209,a_sun
4,1985,1215,a_sun


In [5]:
df_regular.drop(labels=['DayNum', 'WLoc', 'NumOT'], axis=1, inplace=True)
df_regular.head()

,Season,WTeamID,WScore,LTeamID,LScore
0,1985,1228,81,1328,64
1,1985,1106,77,1354,70
2,1985,1112,63,1223,56
3,1985,1165,70,1432,54
4,1985,1192,86,1447,74


In [6]:
df = pd.merge(left=df_regular, right=df_win_team_conference, how='left', on=['Season', 'WTeamID'])
df.head()

,Season,WTeamID,WScore,LTeamID,LScore,WTeamConfAbbrev
0,1985,1228,81,1328,64,big_ten
1,1985,1106,77,1354,70,swac
2,1985,1112,63,1223,56,pac_ten
3,1985,1165,70,1432,54,ivy
4,1985,1192,86,1447,74,ecacm


In [7]:
df_regular = pd.merge(left=df, right=df_loss_team_conference, how='left', on=['Season', 'LTeamID'])
df_regular.head()

,Season,WTeamID,WScore,LTeamID,LScore,WTeamConfAbbrev,LTeamConfAbbrev
0,1985,1228,81,1328,64,big_ten,big_eight
1,1985,1106,77,1354,70,swac,meac
2,1985,1112,63,1223,56,pac_ten,a_sun
3,1985,1165,70,1432,54,ivy,ind
4,1985,1192,86,1447,74,ecacm,ecacm


### Now ```df_regular``` has everything we need. We will first compute each team's winning percentage. ###

In [8]:
grouped_by_Season_WTeamID = df_regular.groupby(['Season', 'WTeamID'])
# grouped_by_Season_WTeamID.LTeamID.count()

In [9]:
df_regular_win_team = grouped_by_Season_WTeamID.LTeamID.count().reset_index(name='W').rename(columns={'WTeamID':'TeamID'})
df_regular_win_team.head()

,Season,TeamID,W
0,1985,1102,5
1,1985,1103,9
2,1985,1104,21
3,1985,1106,10
4,1985,1108,19


In [10]:
grouped_by_Season_LTeamID = df_regular.groupby(['Season', 'LTeamID'])
# grouped_by_Season_LTeamID.LTeamID.count()

In [11]:
df_regular_loss_team = grouped_by_Season_LTeamID.WTeamID.count().reset_index(name='L').rename(columns={'LTeamID':'TeamID'})
df_regular_loss_team.head()

,Season,TeamID,L
0,1985,1102,19
1,1985,1103,14
2,1985,1104,9
3,1985,1106,14
4,1985,1108,6


In [12]:
df_regular_winning_pct_team = pd.merge(left=df_regular_win_team,\
                                       right=df_regular_loss_team,\
                                       how='outer', on=['Season', 'TeamID'])
df_regular_winning_pct_team.fillna(value=0, inplace=True)
df_regular_winning_pct_team.head()

,Season,TeamID,W,L
0,1985,1102,5.0,19.0
1,1985,1103,9.0,14.0
2,1985,1104,21.0,9.0
3,1985,1106,10.0,14.0
4,1985,1108,19.0,6.0


In [13]:
df_regular_winning_pct_team['Pct'] = df_regular_winning_pct_team.W / (df_regular_winning_pct_team.W + df_regular_winning_pct_team.L)
df_regular_winning_pct_team.head()

,Season,TeamID,W,L,Pct
0,1985,1102,5.0,19.0,0.208333
1,1985,1103,9.0,14.0,0.391304
2,1985,1104,21.0,9.0,0.700000
3,1985,1106,10.0,14.0,0.416667
4,1985,1108,19.0,6.0,0.760000


### Next, let's compute each team's average winning margin. ###

For the moment, we accmulate the score difference of wins and losses repectively. Combine accmulated score difference and wins and losses, we have various ways to compute average winning margin.

In [14]:
df_regular.head()

,Season,WTeamID,WScore,LTeamID,LScore,WTeamConfAbbrev,LTeamConfAbbrev
0,1985,1228,81,1328,64,big_ten,big_eight
1,1985,1106,77,1354,70,swac,meac
2,1985,1112,63,1223,56,pac_ten,a_sun
3,1985,1165,70,1432,54,ivy,ind
4,1985,1192,86,1447,74,ecacm,ecacm


In [15]:
df_regular['ScoreDiff'] = df_regular.WScore - df_regular.LScore
df_regular.head()

,Season,WTeamID,WScore,LTeamID,LScore,WTeamConfAbbrev,LTeamConfAbbrev,ScoreDiff
0,1985,1228,81,1328,64,big_ten,big_eight,17
1,1985,1106,77,1354,70,swac,meac,7
2,1985,1112,63,1223,56,pac_ten,a_sun,7
3,1985,1165,70,1432,54,ivy,ind,16
4,1985,1192,86,1447,74,ecacm,ecacm,12


In [16]:
grouped_by_Season_WTeamID = df_regular.groupby(['Season', 'WTeamID'])
# grouped_by_Season_WTeamID.ScoreDiff.agg(np.sum)

In [17]:
df_regular_win_team = grouped_by_Season_WTeamID.ScoreDiff.agg(np.sum).reset_index(name='WAccumScore').rename(columns={'WTeamID':'TeamID'})
df_regular_win_team.head()

,Season,TeamID,WAccumScore
0,1985,1102,50
1,1985,1103,68
2,1985,1104,277
3,1985,1106,95
4,1985,1108,263


In [18]:
grouped_by_Season_LTeamID = df_regular.groupby(['Season', 'LTeamID'])
# grouped_by_Season_LTeamID.ScoreDiff.apply(np.sum)

In [19]:
df_regular_loss_team = grouped_by_Season_LTeamID.ScoreDiff.apply(np.sum).reset_index(name='LAccumScore').rename(columns={'LTeamID':'TeamID'})
df_regular_loss_team.head()

,Season,TeamID,LAccumScore
0,1985,1102,189
1,1985,1103,138
2,1985,1104,43
3,1985,1106,186
4,1985,1108,64


In [20]:
df_regular_winning_margin_team = pd.merge(left=df_regular_win_team,\
                                          right=df_regular_loss_team,\
                                          how='outer', on=['Season', 'TeamID'])
df_regular_winning_margin_team.fillna(value=0, inplace=True)
df_regular_winning_margin_team.head()

,Season,TeamID,WAccumScore,LAccumScore
0,1985,1102,50.0,189.0
1,1985,1103,68.0,138.0
2,1985,1104,277.0,43.0
3,1985,1106,95.0,186.0
4,1985,1108,263.0,64.0


In [21]:
df_regular_results_team = pd.merge(left=df_regular_winning_margin_team,\
                                   right=df_regular_winning_pct_team,\
                                   on=['Season', 'TeamID'],\
                                   how='inner')
df_regular_results_team.head()

,Season,TeamID,WAccumScore,LAccumScore,W,L,Pct
0,1985,1102,50.0,189.0,5.0,19.0,0.208333
1,1985,1103,68.0,138.0,9.0,14.0,0.391304
2,1985,1104,277.0,43.0,21.0,9.0,0.700000
3,1985,1106,95.0,186.0,10.0,14.0,0.416667
4,1985,1108,263.0,64.0,19.0,6.0,0.760000


### Rank conferences by regular season winning percentage and winning margin ###

In [22]:
df_regular.head()

,Season,WTeamID,WScore,LTeamID,LScore,WTeamConfAbbrev,LTeamConfAbbrev,ScoreDiff
0,1985,1228,81,1328,64,big_ten,big_eight,17
1,1985,1106,77,1354,70,swac,meac,7
2,1985,1112,63,1223,56,pac_ten,a_sun,7
3,1985,1165,70,1432,54,ivy,ind,16
4,1985,1192,86,1447,74,ecacm,ecacm,12


In [23]:
df_regular_cross_conference = df_regular[df_regular.WTeamConfAbbrev != df_regular.LTeamConfAbbrev]
df_regular_cross_conference.head()

,Season,WTeamID,WScore,LTeamID,LScore,WTeamConfAbbrev,LTeamConfAbbrev,ScoreDiff
0,1985,1228,81,1328,64,big_ten,big_eight,17
1,1985,1106,77,1354,70,swac,meac,7
2,1985,1112,63,1223,56,pac_ten,a_sun,7
3,1985,1165,70,1432,54,ivy,ind,16
5,1985,1218,79,1337,78,wac,wcc,1


In [24]:
grouped_by_Season_WTeamConfAbbrev = df_regular_cross_conference.groupby(['Season', 'WTeamConfAbbrev'])
df_regular_win_conference = grouped_by_Season_WTeamConfAbbrev.LTeamConfAbbrev.count().reset_index(name='W').rename(columns={'WTeamConfAbbrev':'ConfAbbrev'})
df_regular_win_conference.head()

,Season,ConfAbbrev,W
0,1985,a_sun,37
1,1985,a_ten,34
2,1985,acc,89
3,1985,big_east,73
4,1985,big_eight,61


In [25]:
grouped_by_Season_LTeamConfAbbrev = df_regular_cross_conference.groupby(['Season', 'LTeamConfAbbrev'])
df_regular_loss_conference = grouped_by_Season_LTeamConfAbbrev.WTeamConfAbbrev.count().reset_index(name='L').rename(columns={'LTeamConfAbbrev':'ConfAbbrev'})
df_regular_loss_conference.head()

,Season,ConfAbbrev,L
0,1985,a_sun,42
1,1985,a_ten,40
2,1985,acc,18
3,1985,big_east,16
4,1985,big_eight,33


In [26]:
df_regular_winning_pct_conference = pd.merge(left=df_regular_win_conference,\
                                             right=df_regular_loss_conference,\
                                             how='outer', on=['Season', 'ConfAbbrev'])
df_regular_winning_pct_conference.fillna(value=0, inplace=True)
df_regular_winning_pct_conference['Pct'] = df_regular_winning_pct_conference.W / (df_regular_winning_pct_conference.W + df_regular_winning_pct_conference.L)
df_regular_winning_pct_conference.head()

,Season,ConfAbbrev,W,L,Pct
0,1985,a_sun,37,42,0.468354
1,1985,a_ten,34,40,0.459459
2,1985,acc,89,18,0.831776
3,1985,big_east,73,16,0.820225
4,1985,big_eight,61,33,0.648936


In [27]:
df_regular_win_conference = grouped_by_Season_WTeamConfAbbrev.ScoreDiff.agg(np.sum).reset_index(name='WAccumScore').rename(columns={'WTeamConfAbbrev':'ConfAbbrev'})
df_regular_win_conference.head()

,Season,ConfAbbrev,WAccumScore
0,1985,a_sun,413
1,1985,a_ten,394
2,1985,acc,1564
3,1985,big_east,1046
4,1985,big_eight,922


In [28]:
df_regular_loss_conference = grouped_by_Season_LTeamConfAbbrev.ScoreDiff.agg(np.sum).reset_index(name='LAccumScore').rename(columns={'LTeamConfAbbrev':'ConfAbbrev'})
df_regular_loss_conference.head()

,Season,ConfAbbrev,LAccumScore
0,1985,a_sun,542
1,1985,a_ten,457
2,1985,acc,103
3,1985,big_east,80
4,1985,big_eight,363


In [29]:
df_regular_winning_margin_conference = pd.merge(left=df_regular_win_conference,\
                                                right=df_regular_loss_conference,\
                                                how='outer', on=['Season', 'ConfAbbrev'])
df_regular_winning_margin_conference.fillna(value=0, inplace=True)
df_regular_winning_margin_conference.head()

,Season,ConfAbbrev,WAccumScore,LAccumScore
0,1985,a_sun,413,542
1,1985,a_ten,394,457
2,1985,acc,1564,103
3,1985,big_east,1046,80
4,1985,big_eight,922,363


In [30]:
df_regular_results_conference = pd.merge(left=df_regular_winning_margin_conference,\
                                         right=df_regular_winning_pct_conference,\
                                         on=['Season', 'ConfAbbrev'],\
                                         how='inner')
df_regular_results_conference.head()

,Season,ConfAbbrev,WAccumScore,LAccumScore,W,L,Pct
0,1985,a_sun,413,542,37,42,0.468354
1,1985,a_ten,394,457,34,40,0.459459
2,1985,acc,1564,103,89,18,0.831776
3,1985,big_east,1046,80,73,16,0.820225
4,1985,big_eight,922,363,61,33,0.648936


### Combine feature vector ###
1. Regular season team winning percentage difference
2. Regular season team winning margin difference. winning_margin = (WAccumScore-LAccumScore) / (W+L). This feature needs to be normalized.
3. Regular season conference winning percentage difference.
4. Regular season conference winning margin difference.

In [31]:
df_regular_results_team.head()

,Season,TeamID,WAccumScore,LAccumScore,W,L,Pct
0,1985,1102,50.0,189.0,5.0,19.0,0.208333
1,1985,1103,68.0,138.0,9.0,14.0,0.391304
2,1985,1104,277.0,43.0,21.0,9.0,0.700000
3,1985,1106,95.0,186.0,10.0,14.0,0.416667
4,1985,1108,263.0,64.0,19.0,6.0,0.760000


In [32]:
df_regular_results_team['Margin'] = (df_regular_results_team.WAccumScore - df_regular_results_team.LAccumScore) / (df_regular_results_team.W + df_regular_results_team.L)
df_regular_results_team.head()

,Season,TeamID,WAccumScore,LAccumScore,W,L,Pct,Margin
0,1985,1102,50.0,189.0,5.0,19.0,0.208333,-5.791667
1,1985,1103,68.0,138.0,9.0,14.0,0.391304,-3.043478
2,1985,1104,277.0,43.0,21.0,9.0,0.700000,7.800000
3,1985,1106,95.0,186.0,10.0,14.0,0.416667,-3.791667
4,1985,1108,263.0,64.0,19.0,6.0,0.760000,7.960000


In [33]:
df_regular.head()

,Season,WTeamID,WScore,LTeamID,LScore,WTeamConfAbbrev,LTeamConfAbbrev,ScoreDiff
0,1985,1228,81,1328,64,big_ten,big_eight,17
1,1985,1106,77,1354,70,swac,meac,7
2,1985,1112,63,1223,56,pac_ten,a_sun,7
3,1985,1165,70,1432,54,ivy,ind,16
4,1985,1192,86,1447,74,ecacm,ecacm,12


In [34]:
df_regular_results_wteam = df_regular_results_team[['Season', 'TeamID', 'Pct', 'Margin']].rename(columns={'TeamID':'WTeamID', 'Pct':'WTeamPct', 'Margin':'WTeamMargin'})
df_regular_results_wteam.head()

,Season,WTeamID,WTeamPct,WTeamMargin
0,1985,1102,0.208333,-5.791667
1,1985,1103,0.391304,-3.043478
2,1985,1104,0.700000,7.800000
3,1985,1106,0.416667,-3.791667
4,1985,1108,0.760000,7.960000


In [35]:
df_regular_results_lteam = df_regular_results_team[['Season', 'TeamID', 'Pct', 'Margin']].rename(columns={'TeamID':'LTeamID', 'Pct':'LTeamPct', 'Margin':'LTeamMargin'})
df_regular_results_lteam.head()

,Season,LTeamID,LTeamPct,LTeamMargin
0,1985,1102,0.208333,-5.791667
1,1985,1103,0.391304,-3.043478
2,1985,1104,0.700000,7.800000
3,1985,1106,0.416667,-3.791667
4,1985,1108,0.760000,7.960000


In [36]:
df_win_team_conference.head()

,Season,WTeamID,WTeamConfAbbrev
0,1985,1114,a_sun
1,1985,1147,a_sun
2,1985,1204,a_sun
3,1985,1209,a_sun
4,1985,1215,a_sun


In [37]:
df_regular_results_conference['Margin'] = (df_regular_results_conference.WAccumScore - df_regular_results_conference.LAccumScore) / (df_regular_results_conference.W + df_regular_results_conference.L)
df_regular_results_conference.head()

,Season,ConfAbbrev,WAccumScore,LAccumScore,W,L,Pct,Margin
0,1985,a_sun,413,542,37,42,0.468354,-1.632911
1,1985,a_ten,394,457,34,40,0.459459,-0.851351
2,1985,acc,1564,103,89,18,0.831776,13.654206
3,1985,big_east,1046,80,73,16,0.820225,10.853933
4,1985,big_eight,922,363,61,33,0.648936,5.946809


In [38]:
df_regular_results_wteam_conference = df_regular_results_conference[['Season', 'ConfAbbrev', 'Pct', 'Margin']].rename(columns={'ConfAbbrev':'WTeamConfAbbrev', 'Pct':'WTeamConfPct', 'Margin':'WTeamConfMargin'})
df_regular_results_wteam_conference.head()

,Season,WTeamConfAbbrev,WTeamConfPct,WTeamConfMargin
0,1985,a_sun,0.468354,-1.632911
1,1985,a_ten,0.459459,-0.851351
2,1985,acc,0.831776,13.654206
3,1985,big_east,0.820225,10.853933
4,1985,big_eight,0.648936,5.946809


In [39]:
df_regular_results_lteam_conference = df_regular_results_conference[['Season', 'ConfAbbrev', 'Pct', 'Margin']].rename(columns={'ConfAbbrev':'LTeamConfAbbrev', 'Pct':'LTeamConfPct', 'Margin':'LTeamConfMargin'})
df_regular_results_lteam_conference.head()

,Season,LTeamConfAbbrev,LTeamConfPct,LTeamConfMargin
0,1985,a_sun,0.468354,-1.632911
1,1985,a_ten,0.459459,-0.851351
2,1985,acc,0.831776,13.654206
3,1985,big_east,0.820225,10.853933
4,1985,big_eight,0.648936,5.946809


In [40]:
df_regular_results_wteam.head()

,Season,WTeamID,WTeamPct,WTeamMargin
0,1985,1102,0.208333,-5.791667
1,1985,1103,0.391304,-3.043478
2,1985,1104,0.700000,7.800000
3,1985,1106,0.416667,-3.791667
4,1985,1108,0.760000,7.960000


In [41]:
df_win_team_conference.head()

,Season,WTeamID,WTeamConfAbbrev
0,1985,1114,a_sun
1,1985,1147,a_sun
2,1985,1204,a_sun
3,1985,1209,a_sun
4,1985,1215,a_sun


In [42]:
df_regular_results_wteam_complete =\
    df_regular_results_wteam.merge(df_win_team_conference, on=['Season', 'WTeamID'], how='inner').\
        merge(df_regular_results_wteam_conference, on=['Season', 'WTeamConfAbbrev'], how='inner')
df_regular_results_wteam_complete.head()

,Season,WTeamID,WTeamPct,WTeamMargin,WTeamConfAbbrev,WTeamConfPct,WTeamConfMargin
0,1985,1102,0.208333,-5.791667,wac,0.55914,3.096774
1,1985,1140,0.517241,0.551724,wac,0.55914,3.096774
2,1985,1161,0.571429,1.678571,wac,0.55914,3.096774
3,1985,1218,0.357143,-3.607143,wac,0.55914,3.096774
4,1985,1307,0.586207,3.000000,wac,0.55914,3.096774


In [43]:
df_regular_results_lteam_complete =\
    df_regular_results_lteam.merge(df_loss_team_conference, on=['Season', 'LTeamID'], how='inner').\
        merge(df_regular_results_lteam_conference, on=['Season', 'LTeamConfAbbrev'], how='inner')
df_regular_results_lteam_complete.head()

,Season,LTeamID,LTeamPct,LTeamMargin,LTeamConfAbbrev,LTeamConfPct,LTeamConfMargin
0,1985,1102,0.208333,-5.791667,wac,0.55914,3.096774
1,1985,1140,0.517241,0.551724,wac,0.55914,3.096774
2,1985,1161,0.571429,1.678571,wac,0.55914,3.096774
3,1985,1218,0.357143,-3.607143,wac,0.55914,3.096774
4,1985,1307,0.586207,3.000000,wac,0.55914,3.096774


In [44]:
df_ncaa_tourney = pd.read_csv('DataFiles/NCAATourneyCompactResults.csv')
df_ncaa_tourney.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
0,1985,136,1116,63,1234,54,N,0
1,1985,136,1120,59,1345,58,N,0
2,1985,136,1207,68,1250,43,N,0
3,1985,136,1229,58,1425,55,N,0
4,1985,136,1242,49,1325,38,N,0


In [45]:
df = df_ncaa_tourney[['Season', 'WTeamID', 'LTeamID']]
df.head()

,Season,WTeamID,LTeamID
0,1985,1116,1234
1,1985,1120,1345
2,1985,1207,1250
3,1985,1229,1425
4,1985,1242,1325


In [48]:
df_complete = df.merge(df_regular_results_wteam_complete, on=['Season', 'WTeamID'], how='inner').\
    merge(df_regular_results_lteam_complete, on=['Season', 'LTeamID'], how='inner')

In [49]:
df_features = pd.DataFrame()
df_features['TeamPctDiff'] = df_complete['WTeamPct'] - df_complete['LTeamPct']
df_features['TeamMarginDiff'] = df_complete['WTeamMargin'] - df_complete['LTeamMargin']
df_features['ConfPctDiff'] = df_complete['WTeamConfPct'] - df_complete['LTeamConfPct']
df_features['ConfMarginDiff'] = df_complete['WTeamConfMargin'] - df_complete['LTeamConfMargin']
df_features.head()

,TeamPctDiff,TeamMarginDiff,ConfPctDiff,ConfMarginDiff
0,-0.030303,-6.830303,-0.119048,-4.668367
1,-0.059310,-0.110345,-0.074871,-1.698058
2,-0.145977,-1.943678,0.061907,2.559216
3,0.546616,20.114943,0.534510,19.520599
4,0.098340,7.701149,0.360765,11.705284


### Cells below are initial commit. ###

In [27]:
df_conference_tourney[df_conference_tourney.ConfAbbrev=='ivy']

,Season,ConfAbbrev,DayNum,WTeamID,LTeamID
4406,2017,ivy,131,1343,1335
4407,2017,ivy,131,1463,1217
4408,2017,ivy,132,1343,1463


In [28]:
df_regular_clean.head()

,Season,WTeamID,LTeamID
0,1985,1228,1328
1,1985,1106,1354
2,1985,1112,1223
3,1985,1165,1432
4,1985,1192,1447


In [35]:
grouped_win = df_regular_clean.groupby(['Season', 'WTeamID'])
grouped_loss = df_regular_clean.groupby(['Season', 'LTeamID'])

In [48]:
win = grouped_win.LTeamID.count().reset_index(name='W').rename(columns={'WTeamID':'TeamID'})
win.head()
len(win)

10523

In [49]:
loss = grouped_loss.WTeamID.count().reset_index(name='L').rename(columns={'LTeamID':'TeamID'})
loss.head()
len(loss)

10534

In [57]:
df_regular_win_rate = pd.merge(left=win, right=loss, how='outer', on=['Season', 'TeamID'])
df_regular_win_rate.head()

,Season,TeamID,W,L
0,1985,1102,5.0,19.0
1,1985,1103,9.0,14.0
2,1985,1104,21.0,9.0
3,1985,1106,10.0,14.0
4,1985,1108,19.0,6.0


In [59]:
df_regular_win_rate['WRate'] = df_regular_win_rate.W / (df_regular_win_rate.W + df_regular_win_rate.L)

In [60]:
df_regular_win_rate.head()

,Season,TeamID,W,L,WRate
0,1985,1102,5.0,19.0,0.208333
1,1985,1103,9.0,14.0,0.391304
2,1985,1104,21.0,9.0,0.700000
3,1985,1106,10.0,14.0,0.416667
4,1985,1108,19.0,6.0,0.760000


In [62]:
df_tourney.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
0,1985,136,1116,63,1234,54,N,0
1,1985,136,1120,59,1345,58,N,0
2,1985,136,1207,68,1250,43,N,0
3,1985,136,1229,58,1425,55,N,0
4,1985,136,1242,49,1325,38,N,0


In [63]:
df_tourney_clean = df_tourney.drop(labels=['DayNum', 'WScore', 'LScore', 'WLoc', 'NumOT'], axis=1)
df_tourney_clean.head()

,Season,WTeamID,LTeamID
0,1985,1116,1234
1,1985,1120,1345
2,1985,1207,1250
3,1985,1229,1425
4,1985,1242,1325


In [64]:
df_WTeam_WRate = df_regular_win_rate.rename(columns={'TeamID':'WTeamID', 'WRate':'WTeamWRate'})
df_WTeam_WRate.head()

,Season,WTeamID,W,L,WTeamWRate
0,1985,1102,5.0,19.0,0.208333
1,1985,1103,9.0,14.0,0.391304
2,1985,1104,21.0,9.0,0.700000
3,1985,1106,10.0,14.0,0.416667
4,1985,1108,19.0,6.0,0.760000


In [65]:
df_LTeam_WRate = df_regular_win_rate.rename(columns={'TeamID':'LTeamID', 'WRate':'LTeamWRate'})
df_LTeam_WRate.head()

,Season,LTeamID,W,L,LTeamWRate
0,1985,1102,5.0,19.0,0.208333
1,1985,1103,9.0,14.0,0.391304
2,1985,1104,21.0,9.0,0.700000
3,1985,1106,10.0,14.0,0.416667
4,1985,1108,19.0,6.0,0.760000


In [66]:
df_WTeam_WRate.drop(labels=['W', 'L'], axis=1, inplace=True)
df_WTeam_WRate.head()

,Season,WTeamID,WTeamWRate
0,1985,1102,0.208333
1,1985,1103,0.391304
2,1985,1104,0.700000
3,1985,1106,0.416667
4,1985,1108,0.760000


In [67]:
df_LTeam_WRate.drop(labels=['W', 'L'], axis=1, inplace=True)
df_LTeam_WRate.head()

,Season,LTeamID,LTeamWRate
0,1985,1102,0.208333
1,1985,1103,0.391304
2,1985,1104,0.700000
3,1985,1106,0.416667
4,1985,1108,0.760000


In [68]:
df = pd.merge(left=df_tourney_clean, right=df_WTeam_WRate, how='left', on=['Season', 'WTeamID'])
df.head()

,Season,WTeamID,LTeamID,WTeamWRate
0,1985,1116,1234,0.636364
1,1985,1120,1345,0.620690
2,1985,1207,1250,0.925926
3,1985,1229,1425,0.740741
4,1985,1242,1325,0.766667


In [69]:
df_tourney_with_regular_win_rate = pd.merge(left=df, right=df_LTeam_WRate, how='left', on=['Season', 'LTeamID'])
df_tourney_with_regular_win_rate.head()

,Season,WTeamID,LTeamID,WTeamWRate,LTeamWRate
0,1985,1116,1234,0.636364,0.666667
1,1985,1120,1345,0.620690,0.680000
2,1985,1207,1250,0.925926,0.379310
3,1985,1229,1425,0.740741,0.678571
4,1985,1242,1325,0.766667,0.740741


In [70]:
df_tourney_with_regular_win_rate['WRateDiff'] = df_tourney_with_regular_win_rate.WTeamWRate - df_tourney_with_regular_win_rate.LTeamWRate
df_tourney_with_regular_win_rate.head()

,Season,WTeamID,LTeamID,WTeamWRate,LTeamWRate,WRateDiff
0,1985,1116,1234,0.636364,0.666667,-0.030303
1,1985,1120,1345,0.620690,0.680000,-0.059310
2,1985,1207,1250,0.925926,0.379310,0.546616
3,1985,1229,1425,0.740741,0.678571,0.062169
4,1985,1242,1325,0.766667,0.740741,0.025926


In [71]:
df_wins = pd.DataFrame()
df_wins['WRateDiff'] = df_tourney_with_regular_win_rate['WRateDiff']
df_wins['Result'] = 1

df_losses = pd.DataFrame()
df_losses['WRateDiff'] = -df_tourney_with_regular_win_rate['WRateDiff']
df_losses['Result'] = 0

df_predictions = pd.concat([df_wins, df_losses])
df_predictions.head()

,WRateDiff,Result
0,-0.030303,1
1,-0.059310,1
2,0.546616,1
3,0.062169,1
4,0.025926,1


In [74]:
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.model_selection import GridSearchCV

In [75]:
X_train = df_predictions.WRateDiff.values.reshape(-1,1)
y_train = df_predictions.Result.values
X_train, y_train = shuffle(X_train, y_train)

In [77]:
np.any(np.isnan(X_train))

True

In [78]:
df_predictions[np.isnan(df_predictions.WRateDiff)]

,WRateDiff,Result
406,NaN,1
424,NaN,1
429,NaN,1
435,NaN,1
438,NaN,1
1884,NaN,1
1896,NaN,1
1926,NaN,1
1953,NaN,1
1969,NaN,1


In [79]:
df = df_predictions.dropna(axis=0, how='any')
df[np.isnan(df.WRateDiff)]

,WRateDiff,Result


In [80]:
df.head()

,WRateDiff,Result
0,-0.030303,1
1,-0.059310,1
2,0.546616,1
3,0.062169,1
4,0.025926,1


In [82]:
df_predictions.dropna(axis=0, how='any', inplace=True)
X_train = df_predictions.WRateDiff.values.reshape(-1,1)
y_train = df_predictions.Result.values
X_train, y_train = shuffle(X_train, y_train)

In [83]:
logreg = LogisticRegression()
params = {'C': np.logspace(start=-5, stop=3, num=9)}
clf = GridSearchCV(logreg, params, scoring='neg_log_loss', refit=True)
clf.fit(X_train, y_train)
print('Best log_loss: {:.4}, with best C: {}'.format(clf.best_score_, clf.best_params_['C']))

Best log_loss: -0.6339, with best C: 100.0


In [1]:
X = np.linspace(-1, 1, num=100).reshape(-1, 1)
preds = clf.predict_proba(X)[:,1]

plt.plot(X, preds)
plt.xlabel('Team1 WRate - Team2 WRate')
plt.ylabel('P(Team1 will win)')
plt.show()

NameError: name 'np' is not defined